In [1]:
import pandas as pd
import os
import random
import seaborn as sns; sns.set_theme(color_codes=True)
import pickle

In [2]:
data_dir = '/opt/ml/project/data/' # 경로는 상황에 맞춰서 수정해주세요!
csv_file_path = os.path.join(data_dir, 'total_data.csv') # 데이터는 대회홈페이지에서 받아주세요 :)
df = pd.read_csv(csv_file_path, parse_dates=['Timestamp']) 
df = df.sort_values(by=['userID', 'Timestamp']).reset_index(drop=True)

In [3]:
def get_train_users(df, ratio=0.7, split=True):
    users = list(zip(df['userID'].value_counts().index, df['userID'].value_counts()))
    random.shuffle(users)
    
    max_train_data_len = ratio*len(df)
    sum_of_train_data = 0
    user_ids =[]

    for user_id, count in users:
        sum_of_train_data += count
        if max_train_data_len < sum_of_train_data:
            break
        user_ids.append(user_id)
        
    return user_ids

In [4]:
# DROP ANSWERCODE
test_df = df[df['answerCode'] == -1]
test_users = test_df['userID'].unique()
train_df = df[df['userID'].isin(test_users) == False]

In [10]:
random.seed(42)
train_users = get_train_users(train_df)
print(f'Number of Train Users:',df['userID'].nunique())
print(f'Number of Train Users:',len(train_users))
print(f'Number of Valid Users:',len(set(df['userID'].unique()) - set(test_users) - set(train_users)))
print(f'Number of Test Users:',len(test_users))
users = {'train_users':train_users,'test_users':test_users}

users_file_path = os.path.join(data_dir, f'cv1_users.pickle')
with open(users_file_path,'wb') as f:
    pickle.dump(users, f)

Number of Train Users: 7442
Number of Train Users: 4691
Number of Valid Users: 2007
Number of Test Users: 744
